In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Read the data
X = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [3]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [6]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

In [7]:
# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [8]:
# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [9]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

# Build A Model

### Part A

In [10]:
from xgboost import XGBRegressor

In [11]:
# Define the model
my_model_1 = XGBRegressor(random_state=0)

In [12]:
# Fit the model
my_model_1.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

### Part B

In [14]:
from sklearn.metrics import mean_absolute_error

In [15]:
# Get predictions
predictions_1 = my_model_1.predict(X_valid)

### Part C

In [16]:
mae_1 = mean_absolute_error(predictions_1, y_valid)

In [17]:
# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 17662.736729452055


# Improve Model

In [18]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

In [19]:
# Fit the model
my_model_2.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
# Get predictions
predictions_2 = my_model_2.predict(X_valid)

In [21]:
# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)

In [22]:
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 16688.691513270547


# Break The Model

In [23]:
# Define the model
my_model_3 = XGBRegressor(n_estimators=1)

In [24]:
# Fit the model
my_model_3.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
# Get predictions
predictions_3 = my_model_3.predict(X_valid)

In [26]:
# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

In [27]:
# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)

Mean Absolute Error: 127895.0828807256
